# Growing Subsequences
Given an array of numbers $v_0, v_1, \ldots, v_{n-1}$, find the maximum integer $k$ for which there exists a strictly increasing sequence of indices
$
0 \;\le\; \pi_1 \;<\; \pi_2 \;<\;\dots\;<\;\pi_k \;\le\; n-1
$
satisfying
$
v_{\pi_1} \;\le\; v_{\pi_2} \;\le\;\dots\;\le\; v_{\pi_k}.
$

In [14]:
using BenchmarkTools

# Test
v = [-3, 2, 7, 0, -2, -2, 1]

7-element Vector{Int64}:
 -3
  2
  7
  0
 -2
 -2
  1

In [15]:
# ChatGPT Generated
function brute_force_lnnds_length(v)
    n = length(v)
    best_len = 0
    
    # Check all subsets via bitmask from 0 to 2^n - 1
    for mask in 0:(1 << n) - 1
        current_len = 0
        last_val = typemin(Int)
        valid = true
        
        for i in 0:(n-1)
            if (mask & (1 << i)) != 0
                if v[i+1] >= last_val
                    # Extend this subsequence
                    current_len += 1
                    last_val = v[i+1]
                else
                    # This subset fails the non-decreasing condition
                    valid = false
                    break
                end
            end
        end
        
        # Update best length
        if valid && current_len > best_len
            best_len = current_len
        end
    end
    
    return best_len
end

# Example usage:
max_length = @btime brute_force_lnnds_length(v)
println("Length of the longest non-decreasing subsequence is: ", max_length)


  792.526 ns (0 allocations: 0 bytes)
Length of the longest non-decreasing subsequence is: 4


In [ ]:
function sub(v)
    db = []

    for el in v
        for i in eachindex(db)
            if db[i][2] <= el
                push!(db, (db[i][1] +1, el))
            end
        end

        push!(db, (1, el))
    end

    longest = 0
    for el in db
        if el[1] > longest
            longest = el[1]
        end
    end

    return longest
end

sub(v)

  14.375 μs (28 allocations: 1.27 KiB)


4

# Testing

In [17]:
for size in 2:20
    for _ in 1:10
        v = rand(-10:10, size)
        l_b = brute_force_lnnds_length(v)
        l_l = sub(v)

        if l_b != l_l
            println("\n----")
            println("$v")
            println("$l_b")
            println("$l_l")
            println("----")
        end
    end
end